In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re
from datetime import timedelta

%matplotlib inline

In [3]:
#importing all tables from data folder
members = pd.read_csv('NSS_GreatCare_Members.csv')
admissions = pd.read_csv('NSS_Hospital_Inpatient_Admissions.csv')
pcp = pd.read_csv('NSS_PrimaryCare_Office_Visits.csv', low_memory = False)

2. What percent of patients have a PCP visit within a week of discharge?

In [4]:
#converting admissions dates to datetime for calculations
admissions[['Encounter Start Date','Encounter End Date']] = admissions[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)

#converting pcp dates to datetime for calculations
pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'])

#selecting relevant columns from each data frame, then joining hospital admissions to pcp care
a_df = admissions[['Encounter ID', "Patient ID", 'Encounter Start Date', "Encounter End Date"]]
pcp_df = pcp[['Encounter ID', 'Patient ID', 'Encounter Date']]

#merging relevant admissions info to relevant pcp visits
adpcp_df = a_df.merge(pcp_df, how='left', on='Patient ID')

#creating new column for 7 days, to find out which patients had a PCP visit after a week from discharge
adpcp_df['7_days'] = adpcp_df['Encounter Date'] - adpcp_df['Encounter End Date']
adpcp_df

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_days
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1058 days
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-210 days
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,512 days
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,622 days
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,426 days
...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,8 days
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,16 days
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-558 days
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-860 days


In [5]:
patients7 = []

for i in adpcp_df['7_days']:
    if (i <= dt.timedelta(days=7)) and (i > dt.timedelta(days=0)) :
        patients7.append('yes')
    else:
        patients7.append('no')

In [6]:
adpcp_df['7_days'] = patients7

In [7]:
adpcp_df['7_days'].value_counts()

no     41736
yes     2362
Name: 7_days, dtype: int64

BONUS 5
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?

In [95]:
#finding Provier Group NPI's and counting should provide which facility saw the most patients?? 
#rationale here is that Encounter ID's are unique, so each duplicated NPI should be 1 visit?? 
b1 = pcp[['Encounter ID', 'Provider Group NPI', 'Encounter Date']]
b1['Provider Group NPI'] = b1['Provider Group NPI'].astype('object')
b1['Provider Group NPI'].value_counts()

#make this cleaner somehow to see full PGNPI number

<ipython-input-95-36e80204f589>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b1['Provider Group NPI'] = b1['Provider Group NPI'].astype('object')


1.043427e+09    114849
1.184723e+09     44030
1.164513e+09     18096
1.407059e+09     12041
1.316025e+09      8686
                 ...  
1.962566e+09         1
1.083750e+09         1
1.639473e+09         1
1.528052e+09         1
1.699021e+09         1
Name: Provider Group NPI, Length: 380, dtype: int64

BONUS 5. 
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?

In [82]:
#adding searchable text 'no pcp' to replace blank values in the Provider Group NPI column
b1['Provider Group NPI'] = b1['Provider Group NPI'].fillna('no pcp')

#trying to find how many patients do not have a PCP .. 
no_pcp = []
for index, row in b1.iterrows():
    if row['Provider Group NPI'] == 'no pcp':
        no_pcp.append('No PCP')
    else:
        no_pcp.append('Has PCP')

<ipython-input-82-20e96d26865f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b1['Provider Group NPI'] = b1['Provider Group NPI'].fillna('no pcp')


In [170]:
#3,729 patients do not have a PCP.. drop duplicates because each unique patient id was counted for every itterow calculation?
# seems really low if dupes are dropped.. .
yn = pd.DataFrame(no_pcp)
yn = yn.rename(columns={yn.columns[0]: 'PCP Y/N'})
yn.value_counts()
#yn.drop_duplicates()
#yn

PCP Y/N
Has PCP    227644
No PCP       3729
dtype: int64